In [17]:
from pypdf import PdfReader
import pandas as pd
import re

In [156]:
reader = PdfReader("./749728-physics-9702-november-2025-grade-threshold-table.pdf")
page = reader.pages[1]
text = page.extract_text()
text = (text
        .replace("Option ", "Option")
        .replace("Maximum \nmark after \nweighting ", "Maximum mark after weighting")
        .replace("Combination of \ncomponents ", "Combination of components")
        .replace("A*","\nA*")
        .replace(", ", ",")
        .replace("  ", " "))
text = text.split("\n")
text = text[3:]

date_extractor = reader.pages[0].extract_text()
try: 
    date_extractor = re.search(r"November \d{4}", date_extractor).group(0)
except:
    try:
        date_extractor = re.search(r"June \d{4}", date_extractor).group(0)
    except:
        date_extractor = re.search(r"March \d{4}", date_extractor).group(0)
print(date_extractor)

table_header = text[0:3] + text[3].split(" ")   
table_header.pop(-1)
data = pd.DataFrame(columns=table_header)
data.columns = data.columns.str.lower()

predata = text[4:]
predata.pop(-1)

j = 0
for i in predata:
    i = i.split(" ")
    i.remove("")
    data.loc[j] = i
    j += 1

data["option"] = data["option"] + ' ' + data["combination of components"].str.replace(",", ", ")
data.drop(columns=["combination of components"], inplace=True)
data.set_index("option", inplace=True)
data["date"] = date_extractor
data.replace("–", 0, inplace=True)
data[['maximum mark after weighting', 'a*', 'a', 'b', 'c', 'd', 'e']] = data[['maximum mark after weighting', 'a*', 'a', 'b', 'c', 'd', 'e']].astype(int)

a2_table = data.copy().loc[data["maximum mark after weighting"]>= 131,:]
as_table = data.copy().loc[data["maximum mark after weighting"]< 131,:].replace(0, pd.NA)

display(a2_table)
display(as_table)
display(a2_table.groupby("date").mean().round())

November 2025


,maximum mark after weighting,a*,a,b,c,d,e,date
option,,,,,,,,
"AC 14, 24, 37, 44, 54",260,214,190,166,138,110,83,November 2025
"AX 11, 21, 31, 41, 51",260,207,185,163,136,110,84,November 2025
"AY 12, 22, 33, 42, 52",260,201,177,153,127,101,75,November 2025
"AZ 13, 23, 35, 43, 53",260,207,184,161,134,108,82,November 2025
"BC1 44, 54, 62",260,206,182,158,131,104,77,November 2025
"BC2 44, 54, 63",260,206,181,156,129,102,76,November 2025
"BT1 42, 52, 84",260,194,173,152,127,103,79,November 2025
"BT2 42, 52, 87",260,196,173,150,126,102,78,November 2025
"BU1 43, 53, 85",260,207,185,163,135,108,81,November 2025


,maximum mark after weighting,a*,a,b,c,d,e,date
option,,,,,,,,
"S1 11, 21, 31",130,<NA>,97,85,73,61,49,November 2025
"S3 12, 22, 33",130,<NA>,98,86,73,60,47,November 2025
"S4 12, 22, 34",130,<NA>,98,86,73,61,49,November 2025
"S5 13, 23, 35",130,<NA>,96,83,71,59,47,November 2025
"S6 13, 23, 36",130,<NA>,96,83,71,59,47,November 2025
"SA 14, 24, 37",130,<NA>,100,87,74,62,50,November 2025
"SB 14, 24, 38",130,<NA>,100,87,74,62,50,November 2025


,maximum mark after weighting,a*,a,b,c,d,e
date,,,,,,,
November 2025,260.0,205.0,182.0,159.0,132.0,106.0,80.0
